<a href="https://colab.research.google.com/github/Paraugs/Bachelor/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece
from datasets import load_dataset, Dataset
from transformers import MBartForConditionalGeneration, MBartTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import pandas as pd
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00
PyTorch versija: 2.6.0+cu124
Vai GPU pieejams: True


In [ ]:
from google.colab import files
import json
with open("training_data.json", encoding="utf-8") as f:
    raw_data = json.load(f)
data = []
for item in raw_data:
    source = f"Uzdevums: {item['instruction']}\nStudenta atbilde: {item['input']}"
    target = item["output"]
    data.append({"source": source, "target": target})
from datasets import Dataset
dataset = Dataset.from_list(data)
dataset.shuffle(seed=42).select(range(3))
dataset.shuffle(seed=42).select(range(3)).to_pandas()

Saving training_data.json to training_data (1).json


,source,target
0,Uzdevums: Novērtē studenta funkcionālo prasību...,"Nav skaidrs, kāpēc šij funkcijai ir nepiecieša..."
1,Uzdevums: Novērtē studenta funkcionālo prasību...,Prasība ir saprotama un korekti noformulēta.
2,Uzdevums: Novērtē studenta funkcionālo prasību...,Prasība formulēta korekti un atbilstoši. Nav n...


In [ ]:
from transformers import MBartTokenizer, MBartForConditionalGeneration
LANG_CODE = "lv_LV"
model_name = "facebook/mbart-large-50"
tokenizer = MBartTokenizer.from_pretrained(model_name, src_lang=LANG_CODE, tgt_lang=LANG_CODE)
model = MBartForConditionalGeneration.from_pretrained(model_name)
model.config.forced_bos_token_id = tokenizer.lang_code_to_id[LANG_CODE]
def preprocess(example):
    model_inputs = tokenizer(
        example["source"],
        max_length=512,
        truncation=True,
        padding="longest"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target"],
            max_length=128,
            truncation=True,
            padding="longest"
        )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
tokenized_dataset = tokenized_dataset.filter(lambda example: any(token > 1 for token in example["labels"]))
tokenized_dataset[0]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Map:   0%|          | 0/304 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Filter:   0%|          | 0/304 [00:00<?, ? examples/s]

Piemēru skaits: 304


{'input_ids': [10054,
  18548,
  10321,
  12,
  15360,
  40921,
  3672,
  9836,
  11,
  118345,
  33165,
  22197,
  7660,
  51,
  63028,
  75247,
  5,
  26696,
  11,
  39602,
  13,
  12,
  276,
  20216,
  97607,
  12,
  87,
  13,
  181285,
  18,
  29123,
  48,
  2143,
  29586,
  124011,
  32125,
  99,
  102213,
  7,
  87,
  13,
  2113,
  83074,
  12,
  57,
  102213,
  7,
  140064,
  4,
  52958,
  7,
  48516,
  2541,
  275,
  6636,
  12,
  144194,
  958,
  97398,
  13136,
  48,
  46656,
  203309,
  3519,
  2113,
  83074,
  12,
  171802,
  445,
  2113,
  81231,
  4,
  58841,
  124011,
  9035,
  193,
  169728,
  2,
  250016],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [ ]:
tokenized_dataset = tokenized_dataset.filter(
    lambda example: any(token > 1 for token in example["labels"])
)

Filter:   0%|          | 0/304 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset[0]

Piemēru skaits pēc filtrēšanas: 304


{'input_ids': [10054,
  18548,
  10321,
  12,
  15360,
  40921,
  3672,
  9836,
  11,
  118345,
  33165,
  22197,
  7660,
  51,
  63028,
  75247,
  5,
  26696,
  11,
  39602,
  13,
  12,
  276,
  20216,
  97607,
  12,
  87,
  13,
  181285,
  18,
  29123,
  48,
  2143,
  29586,
  124011,
  32125,
  99,
  102213,
  7,
  87,
  13,
  2113,
  83074,
  12,
  57,
  102213,
  7,
  140064,
  4,
  52958,
  7,
  48516,
  2541,
  275,
  6636,
  12,
  144194,
  958,
  97398,
  13136,
  48,
  46656,
  203309,
  3519,
  2113,
  83074,
  12,
  171802,
  445,
  2113,
  81231,
  4,
  58841,
  124011,
  9035,
  193,
  169728,
  2,
  250016],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = TrainingArguments(
    output_dir="./mbart_retrain",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True,
    logging_steps=10,
    logging_dir="./logs",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-25-7559394149d5>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,5.110800
20,3.664800
30,3.037200
40,2.852600
50,2.699100
60,2.561000
70,2.631700
80,2.296300
90,1.834900
100,1.901700


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250016}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=456, training_loss=1.2032361244946195, metrics={'train_runtime': 301.4206, 'train_samples_per_second': 6.051, 'train_steps_per_second': 1.513, 'total_flos': 525046443737088.0, 'train_loss': 1.2032361244946195, 'epoch': 6.0})

In [ ]:
def generate_comment_debug(student_text, instruction="Novērtē studenta funkcionālo prasību un sniedz komentāru."):
    input_text = f"Uzdevums: {instruction}\nStudenta atbilde: {student_text}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        forced_bos_token_id=tokenizer.lang_code_to_id["lv_LV"],
        early_stopping=True
    )
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
sample_student_answer = "Jānodrošina efektīvā un ātrā veiktspēja (2 minušu laikā), it īpaši, apstrādājot lielus pirkšanas pasūtījumus un piedāvājot lietotājiem labu pieredzi."
generated_comment = generate_comment_debug(sample_student_answer)

Iegūtie tokeni: [2, 250016, 4061, 7, 10564, 193, 57, 18606, 2887, 51, 102735, 14, 110, 5037, 202, 34689, 5, 2]
Dekodētais teksts: Prasība ir saprotama un korekti noformulēta.
Ģenerētais komentārs: Prasība ir saprotama un korekti noformulēta.


In [ ]:
from google.colab import files

Saving training_data.json to training_data (2).json


In [ ]:
import json
from datasets import Dataset
with open("training_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
dataset = Dataset.from_list(raw_data)

In [ ]:
from transformers import MBartTokenizer, MBartForConditionalGeneration
LANG_CODE = "lv_LV"
model_name = "facebook/mbart-large-50"
tokenizer = MBartTokenizer.from_pretrained(model_name, src_lang=LANG_CODE, tgt_lang=LANG_CODE)
model = MBartForConditionalGeneration.from_pretrained(model_name)
model.config.decoder_start_token_id = tokenizer.lang_code_to_id["lv_LV"]
model.config.forced_bos_token_id = tokenizer.lang_code_to_id[LANG_CODE]
def preprocess(example):
    model_inputs = tokenizer(
        example["input"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["output"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)
tokenized_dataset = tokenized_dataset.filter(lambda example: any(token > 1 for token in example["labels"]))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Map:   0%|          | 0/178 [00:00<?, ? examples/s]

Filter:   0%|          | 0/178 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = TrainingArguments(
    output_dir="./mbart_with_context_model",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True,
    logging_steps=10,
    logging_dir="./logs",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-34-f1922865150b>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,10.914000
20,8.916100
30,7.919900
40,6.519800
50,5.323400
60,4.244800
70,3.354600
80,2.382800
90,1.765200
100,1.232800


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250016}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=225, training_loss=2.55731079949273, metrics={'train_runtime': 207.4417, 'train_samples_per_second': 4.29, 'train_steps_per_second': 1.085, 'total_flos': 964372746731520.0, 'train_loss': 2.55731079949273, 'epoch': 5.0})

In [ ]:
def generate_comment_with_context_debug(system_summary, student_text):
    input_text = (
        f"Sistēmas apraksts: {system_summary}\n"
        "Vērtēšanas kritēriji: Mērķis, ievaddati, darbības, izvaddati.\n"
        "Uzdevums: Novērtē studentu atbildi, sniedzot komentāru, kas balstās uz iepriekš minēto informāciju.\n"
        f"Studenta atbilde: {student_text}"
    )
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    output_tokens = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    decoded = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return decoded

In [ ]:
student_text = "Mērķis: Pievienot jauni preci kataloga lai viņus var īre.\nIevaddati: Jaunas preču apraksts  un viņu cena.\nDarbības: Pārbaudīt ir šo prece kataloga vai ne. Ja viņa nav tad pievienot viņu datu bāzei, bet ja viņa ir tad paradīt viņu datu bāzē.\nIzvaddati: Jaunu prece katalogā."
generate_comment_with_context_debug(system_summary, student_text)

Tokeni: [2, 250016, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
Ģenerētais komentārs: 


''

In [ ]:
!pip install evaluate -q

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece evaluate bert-score rouge_score

In [ ]:
import evaluate
bertscore = evaluate.load("bertscore")
rouge = evaluate.load("rouge")
results = [
    {"original_comment": "This is the original comment 1.", "generated_comment": "This is the generated comment 1."},
    {"original_comment": "Another original comment.", "generated_comment": "Another generated comment."},
    {"original_comment": "", "generated_comment": "This generated comment has no original."},
    {"original_comment": "Valid original.", "generated_comment": ""},
]
originals = [entry["original_comment"] for entry in results if entry.get("original_comment", "").strip() and entry.get("generated_comment", "").strip()]
generated = [entry["generated_comment"] for entry in results if entry.get("original_comment", "").strip() and entry.get("generated_comment", "").strip()]
if len(originals) == 0:
else:
    bert_result = bertscore.compute(predictions=generated, references=originals, lang="lv")
    rouge_result = rouge.compute(predictions=generated, references=originals)
    for key, value in rouge_result.items():


=== BERTScore ===
Precision: 0.9125
Recall: 0.9125
F1: 0.9125

=== ROUGE ===
rouge1: 0.75
rouge2: 0.3
rougeL: 0.75
rougeLsum: 0.75


In [ ]:
train_dataset = tokenized_dataset.select(range(140))
eval_dataset = tokenized_dataset.select(range(140, 160))
test_dataset = tokenized_dataset.select(range(160, len(tokenized_dataset)))

In [ ]:
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=3,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_strategy="epoch",
    logging_steps=10,
    report_to="none"
)

In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-47-821055e465ea>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.361900,0.292716
2,0.399400,0.277677
3,0.162600,0.275815


TrainOutput(global_step=105, training_loss=0.3079530988420759, metrics={'train_runtime': 151.578, 'train_samples_per_second': 2.771, 'train_steps_per_second': 0.693, 'total_flos': 455097251266560.0, 'train_loss': 0.3079530988420759, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(test_dataset)

In [ ]:
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

In [ ]:
from evaluate import load
rouge = load("rouge")
bertscore = load("bertscore")
rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
bertscore_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="lv")

In [ ]:
for key, value in rouge_result.items():

=== ROUGE ===
rouge1: 0.0000
rouge2: 0.0000
rougeL: 0.0000
rougeLsum: 0.0000

=== BERTScore ===
F1 vidēji: 0.0000


In [ ]:
def generate_feedback(example):
    inputs = tokenizer(example["source"], return_tensors="pt", truncation=True, padding="longest").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"source": example["source"], "generated_feedback": decoded_output}

sample_data = dataset.shuffle(seed=42).select(range(5))
feedback_results = [generate_feedback(example) for example in sample_data]



import pandas as pd

results_df = pd.DataFrame(feedback_results)
results_df.to_csv("generated_feedback.csv", index=False, encoding="utf-8")
results_df.head()

In [ ]:
def generate_feedback(example):
    inputs = tokenizer(example["source"], return_tensors="pt", truncation=True, padding="longest").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"source": example["source"], "generated_feedback": decoded_output}

sample_data = dataset.shuffle(seed=42).select(range(5))
feedback_results = [generate_feedback(example) for example in sample_data]



import pandas as pd

results_df = pd.DataFrame(feedback_results)
results_df.to_csv("generated_feedback.csv", index=False, encoding="utf-8")
results_df.head()

In [ ]:
from google.colab import files
uploaded_test = files.upload()

with open("model_comments.json", encoding="utf-8") as f:
    test_raw = json.load(f)

test_data = []
for item in test_raw:
    source = f"Uzdevums: {item['instruction']}\nStudenta atbilde: {item['input']}"
    test_data.append({"source": source})
def generate_feedback(example):
    inputs = tokenizer(example["source"], return_tensors="pt", truncation=True, padding="longest").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"source": example["source"], "generated_feedback": decoded_output}
test_feedback_results = [generate_feedback(example) for example in test_data]
import pandas as pd

test_results_df = pd.DataFrame(test_feedback_results)
test_results_df.to_csv("test_feedback_output.csv", index=False, encoding="utf-8")
test_results_df.head()